Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать `pint` + `uncertainties`.

In [25]:
from pint import UnitRegistry
from uncertainties import ufloat, umath
from scipy.constants import k as k_J_per_K
import sympy as sp

In [26]:
ureg = UnitRegistry()

In [27]:
H_отсчета = (ufloat(200, 1) * ureg.kilometer).to_base_units()
H_отсчета

<Quantity(200000+/-1000, 'meter')>

In [28]:
P_отсчета =  (ufloat(8, 1)* 1e-5 * ureg.pascal).to_base_units()
P_отсчета

<Quantity((8.00000+/-1.00000)e-05, 'kilogram / meter / second ** 2')>

In [29]:
H = (ufloat(70, 10) * ureg.kilometer).to_base_units()
H

<Quantity(70000.0+/-10000.0, 'meter')>

In [30]:
σ = (ufloat(8, 2) * ureg.angstrom * ureg.angstrom).to_base_units()
σ

<Quantity((8.00000+/-2.00000)e-20, 'meter ** 2')>

In [31]:
T_отсчета = (ufloat(854.4, 200) * ureg.K).to_base_units()
T_отсчета

<Quantity(854.400+/-200.000, 'kelvin')>

In [32]:
T_экзосферы = (ufloat(1000, 300) * ureg.kelvin).to_base_units()
T_экзосферы

<Quantity(1000.00+/-300.00, 'kelvin')>

Todo: разобраться, почему `ureg` не может сам разобраться с конверсией единиц измерения, и почему там константы типа Units, а не Quantities

In [33]:
k = (k_J_per_K * ureg.J / ureg.K).to_base_units()
k

<Quantity(1.380649e-23, 'kilogram * meter ** 2 / second ** 2 / kelvin')>

In [34]:
p_экз = k * T_экзосферы /  σ / H
p_экз.to('pascal')

<Quantity((2.46544+/-1.02519)e-06, 'pascal')>

In [35]:
h_экз = H_отсчета + H * umath.log( (P_отсчета / p_экз).magnitude )
h_экз.to('km')

<Quantity(443.576+/-53.212, 'kilometer')>

In [36]:
h_экз = h_экз.to_base_units()
h_экз

<Quantity(443576+/-53212, 'meter')>

----

In [37]:
G = (1 * ureg.gravitational_constant).to_base_units()
G

<Quantity(6.6743e-11, 'meter ** 3 / kilogram / second ** 2')>

In [38]:
M_земли = ufloat(5.9722e24, 1.2e21) * ureg.kg
M_земли

<Quantity((5.97220+/-0.00120)e+24, 'kilogram')>

In [39]:
R_земли = (ufloat(6.371e6, 1e4) * ureg.meter).to_base_units()
R_земли

<Quantity((6.37100+/-0.01000)e+06, 'meter')>

In [40]:
R_земли+h_экз

<Quantity((6.81458+/-0.05414)e+06, 'meter')>

In [41]:
v_esc_2 = 2*G*M_земли/(R_земли+h_экз)
v_esc_2

<Quantity((1.16985+/-0.00930)e+08, 'meter ** 2 / second ** 2')>

Разобраться, почему pint не поддерживает корни из квадратичных единиц измерения

In [42]:
v_побега_м_с = umath.sqrt(v_esc_2.magnitude)
v_побега_м_с

10815.97357436471+/-42.981716415365625

Ага, поддерживает, но не через `umath.sqrt`

In [43]:
v_esc = (2*G*M_земли/(R_земли+h_экз))**(1/2)
v_esc

<Quantity(10816.0+/-43.0, 'meter / second')>

In [50]:
v_esc.magnitude

10815.97357436471+/-42.981716415365625

----

In [44]:
m, k, T = sp.symbols('m, k, T', positive=True)  # Mass, Boltzmann constant, temperature
v, theta, phi = sp.symbols('v, theta, phi', real=True)  # Spherical coordinates
m, k, T, v, theta, phi

(m, k, T, v, theta, phi)

In [45]:
норм_коэф = (m / (2 * sp.pi * k * T))**(3/2)
норм_коэф


0.353553390593274*m**1.5/(pi**1.5*T**1.5*k**1.5)

In [46]:
f = норм_коэф * sp.exp(-m * v**2 / (2 * k * T)) * v**2 * sp.sin(theta)
f

0.353553390593274*m**1.5*v**2*exp(-m*v**2/(2*T*k))*sin(theta)/(pi**1.5*T**1.5*k**1.5)

Интегрируем по $\phi$ от $-\pi$ до $\pi$

In [47]:
интеграл_phi = sp.integrate(f, (phi, -sp.pi, sp.pi))
интеграл_phi

0.707106781186548*m**1.5*v**2*exp(-m*v**2/(2*T*k))*sin(theta)/(pi**0.5*T**1.5*k**1.5)

Интегрируем по $\theta$ от $0$ до $\pi$

In [48]:
интеграл_theta = sp.integrate(интеграл_phi, (theta, 0, sp.pi))
интеграл_theta

1.4142135623731*m**1.5*v**2*exp(-m*v**2/(2*T*k))/(pi**0.5*T**1.5*k**1.5)

Интегрируем по $v$ от $v_{esc}$ до $\infty$

In [49]:
рез = sp.integrate(интеграл_theta, (v, v_esc, sp.oo))
рез


SympifyError: SympifyError: "cannot sympify object of type <class 'list'>"

Интегрируем по $\theta$ от $0$ до $\pi$